# Regresión utilizando redes neuronales


Importar librerías necesarias:
+ torch
+ scikit-learn
+ matplotlib
+ seaborn
+ pandas
+ numpy

In [ ]:
import 

Fijar semilla utilizada

In [ ]:
np.random.seed(0)
torch.manual_seed(0)

# Dataset Boston house price
Variables en orden:

+ CRIM tasa de criminalidad per cápita por ciudad
+ ZN proporción de terrenos residenciales zonificados para lotes de más de 25.000 pies cuadrados
+ INDUS proporción de acres de negocios no minoristas por ciudad
+ CHAS variable ficticia de Charles River (= 1 si el terreno limita con el río; 0 en caso contrario)
+ NOX concentración de óxidos nítricos (partes por 10 millones)
+ RM número promedio de habitaciones por vivienda
+ AGE proporción de unidades ocupadas por sus propietarios construidas antes de 1940
+ DIS distancias ponderadas a cinco centros de empleo de Boston
+ RAD índice de accesibilidad a carreteras radiales
+ TAX tasa de impuesto a la propiedad de valor total por cada $10.000
+ PTRATIO relación alumno-maestro por ciudad
+ B 1000(Bk - 0,63)^2 donde Bk es la proporción de poblacion afrodescendiente por ciudad
+ LSTAT % de estatus inferior de la población
+ MEDV Valor medio de las viviendas ocupadas por sus propietarios en miles de dólares

In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :3]])
df = pd.DataFrame(data, columns=columns)
df.head()

Carga de Datos

In [ ]:
data = df[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']].values
target = df['MEDV'].values

print(data.shape)
print(target.shape)

## Exploración de Datos
Realizar al menos:
+ Displot - https://seaborn.pydata.org/generated/seaborn.displot.html
+ Correlation Matrix - https://seaborn.pydata.org/generated/seaborn.heatmap.html

## Escalado de Datos

Para evitar que una variable impacte más que otra, se debe escalar los datos de entrada, en este caso pueden emplear **MinMaxScaler**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)

## Carga de Dataset en Pytorch

Para datos fuera de los ya incluidos en PyTorch, se debe crear el objeto Dataset acorde al entrenamiento.

In [ ]:
from torch.utils.data.dataset import Dataset

class BostonDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

boston_dataset = BostonDataset(data, target)

In [ ]:
data, target = boston_dataset[0]
print(data, target)

In [ ]:
from torch.utils.data import random_split

train_size = int( * len(boston_dataset))
test_size = len(boston_dataset) - train_size
train_dataset, test_dataset = random_split(boston_dataset, [train_size, test_size])

### Dataloaders

A partir del dataset cargado generar un dataloader para entrenamiento y otro para validación.

In [ ]:
batch_size =   # Elegir entre 64-128-256

trainloader = 
valloader = 

# Creación de Modelo
Crear MLP con al menos 2 capas densas y 64 neuronas ocultas.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
      super().__init__()
      # Definir capas
      self.linear1 = nn.Linear(13, 64)


    def forward(self, x):
      x = torch.relu(self.linear1(x))

      return x

model = MLP()

# Seleccion de Optimizador y Funcion de Costo
Cosiderar que la tarea a realizar es Regresión.


In [ ]:
import torch.optim as optim

criterion =  # Que funcion de perdida o criterio emplearias para este problema, revisar documentacion
optimizer =  # Que optimizador o metodo emplearias, revisar documentacion

# Entrenamiento
Completar el código de entrenamiento tomando en consideración los siguientes pasos.

+ Definir la cantidad de epocas a entrenar
+ En un bucle se iteran los datos dentro del DataLoader de entrenamiento
+ Se realiza el paso hacia adelante (o forward) obteniendo la prediccion para el batch
+ Se computa el error obteniendo la funcion de perdida y se retropropaga el error en el paso hacia atras (o backward)
+ Se actualizan los pesos mediante el optimizador en nuestro caso el gradiente descendente
+ Se muestran, almacenan y/o calculan las metricas deseadas

In [ ]:
EPOCHS = 300
train_losses = []
val_losses = []
for epoch in range(EPOCHS):
    train_loss = 0.0
    model.train()
    for inputs, labels in trainloader:
        # setea los gradientes en 0

        # forward

        # backward


        # optimizacion


        # print statistics
        train_loss += loss.item()

    average_train_loss = train_loss / len(trainloader)
    train_losses.append(average_train_loss)

    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for inputs, labels in valloader:


            val_loss += loss.item()

    average_validation_loss = val_loss / len(valloader)
    val_losses.append(average_validation_loss)

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {average_train_loss:.4f}, Validation Loss: {average_validation_loss:.4f}")

print('Entrenamiento finalizado')

Grafique ambas funciones de perdida utilizando matplotlib y comente sus resultados.

In [ ]:
plt.plot(, label='Train Loss')
plt.plot(, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

Guardado de modelo:

Los pesos de los modelos entrenados se pueden almacenar mediante la funcion torch.save()

In [ ]:
torch.save(model.state_dict(), './mlp_boston.pth')

Carga de modelo:

Los pesos almacenados pueden ser cargados junto a la definicion del modelo(clase realizada previamente)

In [ ]:
cnn_loaded = MLP()
cnn_loaded.load_state_dict(torch.load('./mlp_boston.pth', weights_only=True))

# Evaluación de modelo

In [ ]:
prediction = []
labels = []
cnn_loaded.eval()
with torch.no_grad():
    for input, label in valloader:
        outputs = cnn_loaded(input)
        prediction.append(outputs.numpy())
        labels.append(label.numpy())

prediction = np.concatenate(prediction)
prediction = prediction.flatten()
labels = np.concatenate(labels)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, PredictionErrorDisplay

mse = mean_squared_error(labels, prediction)
mae = mean_absolute_error(labels, prediction)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse:.4f},MSE: {mse:.4f}, MAE: {mae:.4f}")

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PredictionErrorDisplay.from_predictions(
    labels,
    y_pred=prediction,
    kind="actual_vs_predicted",
    subsample=100,
    ax=axs[0],
    random_state=0,
)
axs[0].set_title("Valores reales vs. Valores predichos")
PredictionErrorDisplay.from_predictions(
    labels,
    y_pred=prediction,
    kind="residual_vs_predicted",
    subsample=100,
    ax=axs[1],
    random_state=0,
)
axs[1].set_title("Residuos vs. Valores Predichos")
fig.suptitle("Gráficos de evaluación de predicciones")
plt.tight_layout()
plt.show()